In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from pathlib import Path
import scienceplots

plt.style.use(['science', 'bright'])
plt.rcParams['figure.dpi'] = 300

In [18]:
config = {
    'data_dir': Path('data/noise/'),
    'models_dir': Path('results/models/'),
    'num_cosmologies': 10,
    'realizations_per_cosmology': 50,
    'lmax': 2400,  # 2401 puntos (0-2400)
    'param_names': ['Omega_m', 'Omega_b', 'h', 'sigma_8', 'ns', 'tau']
}

In [19]:
# %%
class CosmologicalNetwork(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, hidden_units, dropout_rate):
        super().__init__()
        
        layers = []
        layers.append(torch.nn.Linear(input_size, hidden_units))
        layers.append(torch.nn.ReLU())
        layers.append(torch.nn.Dropout(dropout_rate))
        
        for _ in range(hidden_layers - 1):
            layers.append(torch.nn.Linear(hidden_units, hidden_units))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(dropout_rate))
        
        layers.append(torch.nn.Linear(hidden_units, output_size))
        
        self.network = torch.nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

In [20]:
def load_model(model_path):
    """Carga un modelo entrenado"""
    checkpoint = torch.load(model_path, weights_only=False)
    
    model = CosmologicalNetwork(
        input_size=config['lmax'] + 1,
        output_size=len(config['param_names']),
        hidden_layers=checkpoint['best_params']['hidden_layers'],
        hidden_units=checkpoint['best_params']['hidden_units'],
        dropout_rate=checkpoint['best_params']['dropout_rate']
    )
    
    model.load_state_dict(checkpoint['model_state_dict'])
    return model, checkpoint

In [21]:
# %%
# Cargar espectros y parámetros
spectra = torch.load(config['data_dir'] / 'cmb_observed_TTnoise.pt', weights_only=True)
params = torch.load(config['data_dir'] / 'sim_params.pt', weights_only=True)

print(f"Espectros: {spectra.shape} (realizaciones × lmax)")
print(f"Parámetros: {params.shape} (realizaciones × parámetros)")

Espectros: torch.Size([500, 2401]) (realizaciones × lmax)
Parámetros: torch.Size([500, 6]) (realizaciones × parámetros)


In [22]:
# %%
def evaluate_single_model(fold_number=0):
    """Evalúa un modelo específico"""
    # Cargar modelo
    model_path = config['models_dir'] / f'model_fold_{fold_number}.pth'
    model, checkpoint = load_model(model_path)
    
    # Obtener realizaciones de prueba (cosmología excluida)
    test_start = fold_number * config['realizations_per_cosmology']
    test_end = test_start + config['realizations_per_cosmology']
    test_idx = range(test_start, test_end)
    
    X_test = spectra[test_idx].numpy()
    y_test = params[test_idx].numpy()
    
    # Normalizar datos de entrada
    X_test_norm = (X_test - checkpoint['spectra_mean'].numpy()) / (checkpoint['spectra_std'].numpy() + 1e-8)
    
    # Predecir
    model.eval()
    with torch.no_grad():
        predictions = model(torch.FloatTensor(X_test_norm)).numpy()
    
    # Desnormalizar predicciones
    predictions = predictions * checkpoint['params_std'].numpy() + checkpoint['params_mean'].numpy()
    
    # Calcular métricas
    mse = mean_squared_error(y_test, predictions, multioutput='raw_values')
    errors = (predictions - y_test) / y_test * 100  # Error porcentual
    
    # Visualización
    plt.figure(figsize=(15, 10))
    for i, name in enumerate(config['param_names']):
        plt.subplot(2, 3, i+1)
        plt.scatter(y_test[:, i], predictions[:, i], alpha=0.6)
        plt.plot([y_test[:, i].min(), y_test[:, i].max()], 
                 [y_test[:, i].min(), y_test[:, i].max()], 'r--')
        plt.title(f'{name}\nMSE: {mse[i]:.2e}')
        plt.xlabel('Valor Real')
        plt.ylabel('Predicción')
    plt.tight_layout()
    plt.show()
    
    # Mostrar estadísticas
    print(f"\nResultados para modelo fold {fold_number} (excluyendo cosmología {fold_number}):")
    print("Error porcentual medio ± desviación estándar:")
    for i, name in enumerate(config['param_names']):
        print(f"{name}: {np.mean(np.abs(errors[:, i])):.2f}% ± {np.std(errors[:, i]):.2f}%")
    
    return predictions, y_test, errors

In [23]:
# Ejemplo: evaluar el primer fold (excluyendo cosmología 0)
predictions, true_values, errors = evaluate_single_model(fold_number=0)

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'